# Analysis of Lipid Bilayer Dataset

Phospholipid bilayers represent a helpful model description for cell membranes. 
The phospholipid defines the interface between the inside and outside of the cell, and reflectometry techniques are well-suited to studying these systems. 
However, to get the most from our reflectometry data, we must produce an accurate and descriptive model of the experimental system. 
We will use the Python library [`refnx`](https://refnx.readthedocs.io/en/latest/index.html) and this Jupyter Notebook to build this model. 

<img src='./assets/bilayer.png' width='30%'><br>
<small>
A pictorial description of a phospholipid bilayer.
</small>

## Intended Learning Outcomes

- Use the `refnx.reflect.LipidBilayer` to analyse multiple contrast neutron reflectometry data from a lipid bilayer at the solid-liquid interface. 
- Contextualise the implementation of the `refnx.reflect.LipidBilayer` class in the multilayer interpretation of reflectometry analysis
- Assess how neutron reflectometry could be used to analyse your systems of interest. 

This example has been taken from the [`refnx` documentation](https://refnx.readthedocs.io/en/latest/lipid.html) that was developed by [Andrew Nelson](https://www.ansto.gov.au/people/dr-andrew-nelson) (ANSTO).

## Description of the data

The data you will be looking at was collected at the Platypus reflectometer at the ANSTO reactor source in Australia. 
The data are stored as four columns ASCII files, with the columns representing the *q*-vectors, the measured reflectivity, the uncertainty in the measured reflectivity, and the width of a Gaussian resolution function. 
Files are present from three measurements. 
The system that was measured consists of a silicon slab, a bilayer of DMPC (1,2-dimyristoyl-*sn*-glycero-3-phosphocholine), and water arranged as shown below. 
The neutrons enter the sample through the silicon. 

```
|--Silicon--|
|-----------|
|--Bilayer--|
|-----------|
|---Water---|
```

The three measurements were obtained by changing the hydrogen/deuterium ratio in the water. 
It is assumed that this has no effect on the structure of the bilayer and, therefore, offers multiple measurements of the same structural system that can be co-refined. 

Let's start with reading the data and plotting it. 
We will use the `ReflectDataset` class from `refnx` to do this. 

In [ ]:
import matplotlib.pyplot as plt
from refnx.dataset import ReflectDataset

data_d2o = ReflectDataset('./data//bilayer_d2o.dat')
data_hdmix = ReflectDataset('./data/bilayer_hdmix.dat')
data_h2o = ReflectDataset('./data/bilayer_h2o.dat')

fig, ax = plt.subplots()
data_d2o.plot(fig)
data_hdmix.plot(fig)
data_h2o.plot(fig)
ax.set_xlabel('$q$/Å')
ax.set_ylabel('$R(q)$')
ax.set_yscale('log')
plt.show()

The three datasets are noticeably different despite coming from the same structural system. 
This is the effect of changing from pure D<sub>2</sub>O to a mixture of D<sub>2</sub>O and H<sub>2</sub>O to pure H<sub>2</sub>O.

## Building the Model

Having seen the data, the next aim is to build the structural model for the system. 
As shown above, in the sample description, the bilayer is assembled on a silicon block. 
Therefore, we need an object of type `SLD` that can describe the silicon. 

In [ ]:
from refnx.reflect import SLD

si = SLD(2.07, name='Si')
print(si)

#### Task: Create the remaining materials

The silicon block has a naturally formed layer of SiO<sub>2</sub> on it. Therefore, we need an `SLD` object for SiO<sub>2</sub> and the three isotopically different forms of water used. 
Create the four other `SLD` objects in the empty notebook cell below, and use the variable names given in the table. 

| Material    | SLD (10 <sup>-6</sup> Å<sup>-2</sup>) |
| -------- | ------- |
| `sio2`  | 3.47    |
| `d2o` | 6.36    |
| `hdmix`    | 2.07  |
| `h2o` | -0.56 | 

In [ ]:
### ANSWER

sio2 = SLD(3.4, name='SiO2')
d2o = SLD(6.36, name='D2O')
hdmix = SLD(2.07, name='HD-Mix')
h2o = SLD(-0.56, name='H2O')

These objects are all parameters that can be refined in the model-fitting process. 
We can see if a parameter can vary with the following command (notice that since the `SLD` object may include an imaginary component, it in fact contains two parameters). 

In [ ]:
d2o.real.vary

Currently, the real SLD of the `d2o` object cannot vary. 
This can be changed with the `setp` method. 

In [ ]:
d2o.real.setp(vary=True, bounds=(6.1, 6.36))
d2o.real.vary

This will ensure that in the optimisation, the value of `d2o` is constrained between the values of 6.1 &times;10 <sup>-6</sup> Å<sup>-2</sup> and 6.36 &times;10 <sup>-6</sup> Å<sup>-2</sup>. 

The next step will be to create a layer of the SiO<sub>2</sub> material. 
To do this, we call the `sio2` object with two values: the first is the thickness of the layer, and the second is the roughness between this layer and the layer above it (for the SiO<sub>2</sub> layer this is the pure silicon layer) in ångströms.

In [ ]:
sio2_layer = sio2(15, 3)
sio2_layer.name = 'SiO2 Layer'
print(sio2_layer)

#### Task: Varying Parameters of the SiO<sub>2</sub> Layer

You should notice that the `sio2_layer` object has a few parameters: thickness, roughness, and solvent volume fraction. 
Similar to the SLD, these can be allowed to vary. 
Set the `sio2_layer` parameters in the table to vary between the given bounds in the cell below.

| Parameter | Min | Max |
| -------- | ------- | ------- |
| `thick` | 2 | 30 |
| `rough` | 0 | 7 |
| `vfsolv` | 0.0 | 0.5 | 

In [ ]:
### ANSWER

sio2_layer.thick.setp(vary=True, bounds=(2, 30))
sio2_layer.rough.setp(vary=True, bounds=(0, 7))
sio2_layer.vfsolv.setp(vary=True, bounds=(0, 0.5))

The next step is to create the `refnx.reflect.LipidLeaflet` objects (note that two will be required, one for each layer in the bilayer). 
This imposes some physical constraints on our model, specifically that the number of head groups in the lipid layers must equal the number of pairs of tail groups. 
This constraint makes logical sense, as there are covalent chemical bonds between the phospholipid components. 
Let's look at the input requirements for the `refnx.reflect.LipidLeaflet`. 

In [ ]:
from refnx.reflect import LipidLeaflet

LipidLeaflet?

There are a large number of required inputs, so let's build these one by one. 
The first is the `apm` or area per molecule; this is the area that each lipid occupies in the lipid layer. 
We will create this directly as a `Parameter`. 

In [ ]:
from refnx.analysis import Parameter

apm = Parameter(56, name='area per molecule', vary=True, bounds=(52, 65))
print(apm)

#### Task: Remaining Parameter Creation 

Using the `Parameter` function, you should create the remaining parameters (given in the table below) for the `LipidLeafet` in the cell below. 
Where no min and max value is given, the parameter should not be able to vary. 
We will then discuss what each of these parameters means.

| Parameter | Initial Value | Min | Max |
| -------- | ------- | ------- | ------- |
| `b_heads` | 6.01&times;10<sup>-4</sup> | -- | -- |
| `b_tails` | -2.92&times;10<sup>-4</sup> | -- | -- |
| `v_heads` | 319 | -- | -- |
| `v_tails` | 782 | -- | -- |
| `inner_head_thickness` | 9 | 4 | 11 |
| `outer_head_thickness` | 9 | 4 | 11 |
| `tail_thickness` | 14 | 10 | 17 |
| `solv_roughness` | 3 | 0 | 5 |

In [ ]:
### ANSWER
b_heads = Parameter(6.01e-4, 'b_heads')
b_tails = Parameter(-2.92e-4, 'b_tails')

v_heads = Parameter(319, 'v_heads')
v_tails = Parameter(782, 'v_tails')

inner_head_thickness = Parameter(9, 'inner_head_thickness', vary=True, bounds=(4, 11))
outer_head_thickness = Parameter(9, 'outer_head_thickness', vary=True, bounds=(4, 11))
tail_thickness = Parameter(14, 'tail_thickness', vary=True, bounds=(10, 17))
solv_roughness = Parameter(3, 'solv_roughness', vary=True, bounds=(0, 5))

These parameters are the following:
- `b_heads` and `b_tails` are the scattering lengths of the heads and tails (in ångströms), respectively.
- `v_heads` and `v_tails` are the volume occupied by the heads and tails (in ångströms<sup>3</sup>), respectively.
- `inner_head_thickness` is the head layer's thickness next to the silicon block.
- `outer_head_thickness` is the same parameter as the head layer in the water.
- `tail_thickness` is the tail layer thickness. 

The two `LipidLeaflet` can now be created, with roughness values of 3 Å.

In [ ]:
inner_leaflet = LipidLeaflet(apm,
                             b_heads, v_heads, inner_head_thickness,
                             b_tails, v_tails, tail_thickness,
                             3, 3)

outer_leaflet = LipidLeaflet(apm,
                             b_heads, v_heads, outer_head_thickness,
                             b_tails, v_tails, tail_thickness,
                             3, 0, reverse_monolayer=True)

Note that the monolayer is reversed for the `outer_leaflet` so that the tails are in contact between the two layers. 

The final step in building the model is to combine all the layers for each of the three isotopic contrasts. 

In [ ]:
s_d2o = si | sio2_layer | inner_leaflet | outer_leaflet | d2o(0, solv_roughness)
s_hdmix = si | sio2_layer | inner_leaflet | outer_leaflet | hdmix(0, solv_roughness)
s_h2o = si | sio2_layer | inner_leaflet | outer_leaflet | h2o(0, solv_roughness)

The `si` layer has no roughness as it is the top layer in the model. 

## Calculating Reflectometry from the Models

With the model created, we now need to add some reflectometry parameters, the scaling of the data and the background level (only the `s_d2o` system requires a scale change). 
This is done with the `refnx.reflect.ReflectModel` objects. 

In [ ]:
from refnx.reflect import ReflectModel

model_d2o = ReflectModel(s_d2o)
model_hdmix = ReflectModel(s_hdmix)
model_h2o = ReflectModel(s_h2o)

model_d2o.scale.setp(vary=True, bounds=(0.9, 1.1))

model_d2o.bkg.setp(vary=True, bounds=(-1e-6, 1e-6))
model_hdmix.bkg.setp(vary=True, bounds=(-1e-6, 1e-6))
model_h2o.bkg.setp(vary=True, bounds=(-1e-6, 1e-6))

Finally, an `Objective` is created, similar to the previous example. 
However, one is created for each system this time, and these are merged into a `GlobalObjective`.

In [ ]:
from refnx.analysis import Objective, GlobalObjective

objective_d2o = Objective(model_d2o, data_d2o)
objective_hdmix = Objective(model_hdmix, data_hdmix)
objective_h2o = Objective(model_h2o, data_h2o)

global_objective = GlobalObjective([objective_d2o, objective_hdmix, objective_h2o])

Before we perform the fitting, we can look at the current agreement between the model and the data. 

In [ ]:
fig, ax = plt.subplots()
global_objective.plot(fig=fig)
ax.set_xlabel('$q$/Å')
ax.set_ylabel('$R(q)$')
ax.set_yscale('log')
plt.show()

The `GlobalObjective` acts the same way as the `Objective` objects. 
Therefore, they can be passed to the `CurveFitter`, and a fitting algorithm may be applied. 

In [ ]:
from refnx.analysis import CurveFitter

fitter = CurveFitter(global_objective)
fitter.fit('differential_evolution')

With the fitting complete, we check that the fit has improved. 

In [ ]:
fig, ax = plt.subplots()
global_objective.plot(fig=fig)
ax.set_xlabel('$q$/Å')
ax.set_ylabel('$R(q)$')
ax.set_yscale('log')
plt.show()

We can also look at the optimised values of the varying parameters. 

In [ ]:
print(global_objective.varying_parameters())

We can look at the resulting scattering length density profiles. 

In [ ]:
fig, ax = plt.subplots()
ax.plot(*s_d2o.sld_profile(), label='d2o')
ax.plot(*s_hdmix.sld_profile(), label='hdmix')
ax.plot(*s_h2o.sld_profile(), label='h2o')
ax.set_ylabel("$\\rho$ / $10^{-6}$ Å$^{-2}$")
ax.set_xlabel("z / Å")
ax.legend()
plt.show()

#### Task: Perform MCMC on the Bilayer System

In the previous exercise, you performed Markov chain Monte Carlo (MCMC) sampling to investigate the system's posterior distribution. 
Perform the same operation and plot the corner plot for the posterior distribution. 
*Note that this may be very slow on the Binder resource and is probably best run locally.

In [ ]:
### ANSWER

fitter.sample(400)
fitter.sampler.reset()
fitter.sample(30, nthin=100)

global_objective.corner()
plt.show()

#### Task: Consider how reflectometry may be used in your research

How may the reflectometry be used in your research? 
Design a potential reflectometry experiment if you think it would be helpful for your research. 
Contact an instrument scientist and assess, with them, the feasibility of the measurements/analysis. 